Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'AMZN'

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo('AMZN', '2020-01-01', '2021-11-01')
df = df.iloc[: , :4]
#df = df[df.columns[:4]]
df = df.dropna()
#df.head()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2019-12-31,1842.000000,1853.260010,1832.229980,1847.839966
2020-01-02,1875.000000,1898.010010,1864.150024,1898.010010
2020-01-03,1864.500000,1886.199951,1864.500000,1874.969971
2020-01-06,1860.000000,1903.689941,1860.000000,1902.880005
2020-01-07,1904.500000,1913.890015,1892.040039,1906.859985
...,...,...,...,...
2021-10-25,3335.000000,3347.800049,3297.699951,3320.370117
2021-10-26,3349.510010,3416.120117,3343.979980,3376.070068
2021-10-27,3388.000000,3437.000000,3371.449951,3392.489990


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window = 15).mean()

In [5]:
df['Open_1_Change'] = df['Open'] - df['Open'].shift(1)  #df['Open'].diff()

In [6]:
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2020-01-22,1896.089966,1902.500000,1883.339966,1887.459961,1883.441325,31.089966
2020-01-23,1885.109985,1889.979980,1872.760010,1884.579956,1885.890658,-10.979980
2020-01-24,1891.369995,1894.989990,1847.439941,1861.640015,1883.465991,6.260010
2020-01-27,1820.000000,1841.000000,1815.339966,1828.339966,1880.357324,-71.369995
2020-01-28,1840.500000,1858.109985,1830.020020,1853.250000,1877.048657,20.500000
...,...,...,...,...,...,...
2021-10-25,3335.000000,3347.800049,3297.699951,3320.370117,3330.515365,-86.000000
2021-10-26,3349.510010,3416.120117,3343.979980,3376.070068,3340.853369,14.510010
2021-10-27,3388.000000,3437.000000,3371.449951,3392.489990,3349.552035,38.489990


In [7]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(df['Close'].shift(-1) - df['Close'] >= 0, 1, -1)
y

array([-1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1,  1, -1, -1,
        1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1, -1,  1, -1,
       -1,  1, -1,  1,  1,  1, -1,  1,  1, -1,  1, -1,  1, -1, -1,  1, -1,
        1,  1,  1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1, -1, -1,  1,
        1, -1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1,
       -1, -1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,
        1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1,  1,  1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1,  1,  1,  1, -1,  1,  1,
        1, -1, -1, -1,  1, -1, -1,  1,  1, -1,  1, -1,  1,  1,  1, -1,  1,
        1,  1,  1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1,
        1,  1,  1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1, -1, -1, -1,
       -1,  1, -1, -1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1, -1, -1, -1,
        1, -1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,
       -1,  1, -1, -1,  1

In [8]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.
#X_train,y_train 70% 
#make prediction on the x_test and compare them with y_test
index = int(0.7*len(X))
X_train, X_test, y_train, y_test = X[:index], X[index:], y[:index], y[index:] #X[:index] represent the first 70%, X[index:] represent the rest 30%

In [9]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.
#fit the model to the training sets.
logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

In [10]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[-0.008586618589872863]
1,High,[0.006425748879298361]
2,Low,[0.009618948270876117]
3,Close,[-0.0076807413793991385]
4,Close_15_Rolling,[0.0003491822761722174]
5,Open_1_Change,[-0.0023921980645973877]


In [11]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [12]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_test, predictions)

array([[ 2, 64],
       [ 4, 65]])

In [13]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test, y_test)

0.4962962962962963

In [14]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val =  cross_val_score(logistic ,X_test, y_test)
cross_val.mean()

0.47407407407407404